## Importing Libraries

In [1]:
# importing libraries
# data manipulation
import pandas as pd
#modeling
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#visualization
import matplotlib.pyplot as plt
import seaborn as sns



## Reading the data

In [ ]:
# load the data
df = pd.read_csv('mpesa_data.csv')

## Basic Data Exploration

In [ ]:
# top of the data
df.head(10)

In [ ]:
# bottom of the data
df.tail(10)

In [ ]:
# number of rows and columns
df.shape

In [ ]:
df.info()

## Data Cleaning

In [ ]:
# check for columns that do not have data
empty_columns = df.columns[df.isna().all()].to_list()
print('Columns with no data:', empty_columns)

In [ ]:
# check for missing values in each column
print("Missing values in each column:")
print(df.isnull().sum())

In [ ]:
# Create a copy of the original DataFrame to avoid modifying the CSV
data = df.copy()

In [ ]:
# replace missing values in "Paid In" and "Withdrawn" with 0
data['Paid In'] = data['Paid In'].fillna(0)
data['Withdrawn'] = data['Withdrawn'].fillna(0)

# check data
print(data.isnull().sum())

In [ ]:
# replace missing values in "Details" with 'Other'
data['Details'] = data['Details'].fillna('Other')
print(data.isnull().sum())

In [ ]:
# check if any data has been lost
print(data.shape)

In [ ]:
# check for any duplicated rows
print("Number of duplicated rows:", data.duplicated().sum())

## Feature Engineering

In [ ]:
#1. classify transactions as 'Withdraw' (1) if Withdrawn == 0.0, else 'Paid in' (0)

data['Type'] = data['Withdrawn'].apply(lambda x: 1 if x==0.0 else 0) 

In [ ]:
data.tail(10)

In [ ]:
#2. function to categorize the transaction details
def category_details(details):

    # convert the details to lowercase
    details = str(details).lower()

    if any(word in details for word in ["airtime", "tingg", "safaricom", "airtel", "bundles", "gessy"]):
        return "Airtime"
 
    elif any(word in details for word in ["kplc"]):
        return "Power"
    elif any(word in details for word in ["7629905"]):
        return "Rent Payment"

    elif any(word in details for word in ["cleanshelf", "equity", "kcb","naivas", "tuskys", "quick mart", "carrefour", "4093275","supermarket", "shopping", "small business", "mall","jumia", "kilimall", "amazon", "shop", "market", "merchant", "direct pay"]):
        return "Shopping"
    elif any(word in details for word in ["baraka", "java", "hotel", "restaurant", "cafe"]):
        return "Restaurant"
    elif any(word in details for word in ["sacco", "uber"]):
        return "Transport"
    elif any(word in details for word in ["alpha", "water"]):
        return "Water"
    elif any(word in details for word in ["butchery", "meat", "butcher"]):
        return "Butchery"
    elif any(word in details for word in ["customer transfer"]):
        return "People Transfer"
    elif any(word in details for word in ["withdraw"]):
        return "Withdrawals"
    elif any(word in details for word in ["charge"]):
        return "Transaction Charge"

    elif any(word in details for word in ["pay bill"]):
        return "Pay Bill"
    else:
        return "Other"

In [ ]:
# apply the function on the "Details" column
data['Category'] = data['Details'].apply(category_details)

In [ ]:
# check the output
data.tail(10)

In [ ]:
# check value counts in each category
data['Category'].value_counts()

In [ ]:
'''remove the transactions that were paid into your account and only remain with transactions withdrawn from account, 
because we are interested in the spending pattern'''

df= data[data['Type'] == 0].copy()
df.head()

In [ ]:
df.shape

In [ ]:
# Extract the hour, day and month from the time column
df['Hour'] = pd.to_datetime(df['Completion Time']).dt.hour

df['DayOfWeek'] = pd.to_datetime(df['Completion Time']).dt.day_name()

df['Month'] = pd.to_datetime(df['Completion Time']).dt.month

df['MonthName'] = pd.to_datetime(df['Completion Time']).dt.month_name()

df.head(5)

In [ ]:
# Calculate the net movement (amount)
df['Amount'] = df['Paid In'] - df['Withdrawn']
df.head()

In [ ]:
# one-hot encoding to the category column
category_dummies = pd.get_dummies(df['Category'], prefix='Category')
# convert the dummies to integer type and add a prefix
category_dummies = category_dummies.astype(int).add_prefix('Converted_')

In [ ]:
# concatenate the category dummies with the original dataframe
df_clean = pd.concat([df, category_dummies], axis=1)

In [ ]:
df_clean.head()

In [ ]:
# one-hot encoding on the time columns
# day of the week
dayofweek_dummies = pd.get_dummies(df_clean['DayOfWeek'], prefix='DayOfWeek')
dayofweek_dummies = dayofweek_dummies.astype(int).add_prefix('Converted_')

# concatenate the dayofweek_dummies to original df
df_clean = pd.concat([df_clean, dayofweek_dummies], axis=1)

# hour
hour_dummies = pd.get_dummies(df_clean['Hour'], prefix='Hour')
hour_dummies = hour_dummies.astype(int).add_prefix('Converted_')

# concatenate the hour_dummies to original df
df_clean = pd.concat([df_clean, hour_dummies], axis=1)

# month
month_dummies = pd.get_dummies(df_clean['Month'], prefix='Month')
month_dummies = month_dummies.astype(int).add_prefix('Converted_')

# concatenate original df with month_dummies
df_clean = pd.concat([df_clean, month_dummies], axis=1)

In [ ]:
df_clean.head()

## MODELLING

In [ ]:
# extract the encoded columns
encoded_columns = [col for col in df_clean.columns if col.startswith('Converted_')]
encoded_columns

In [ ]:
# combine the features into a single dataframe including "Amount" column
clustering_data = pd.concat([df_clean[encoded_columns], df_clean[['Amount']]], axis=1)
clustering_data.head()

In [ ]:
# check the datatypes on the data
clustering_data.dtypes

In [ ]:
# scale the amount column
scaler = StandardScaler()
clustering_data['amount'] = scaler.fit_transform(clustering_data[['Amount']])
clustering_data.head()

In [ ]:
# determine the optimum number of clusters using silhoutte method
silhouette_scores = []
cluster_range = range(2,11) # test cluster sizes 2 - 10

for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(clustering_data)
    silhouette_avg = silhouette_score(clustering_data, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# plot the silhouette scores
plt.figure(figsize=(12, 6))
plt.plot(cluster_range, silhouette_scores, marker = 'o')
plt.title('Silhouette Scores for Different Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('silhouette Score')
plt.grid(True)
plt.show()

# find optimum number of clusters
optimal_clusters = cluster_range[silhouette_scores.index(max(silhouette_scores))]
print(f'Optimal number of clusters is:', optimal_clusters)

In [ ]:
# fit the model with the optimum number of clusters
model = KMeans(n_clusters=optimal_clusters, random_state=42)
df_clean['purpose_cluster'] = model.fit_predict(clustering_data) # make cluster predictions with the model

In [ ]:
df_clean.head()

In [ ]:
# check model performance (silhouette score)
from sklearn.metrics import silhouette_score

X_scaled = clustering_data.values #Use the scaled data for silhouette score calculation
labels = model.predict(X_scaled)

# calculate the silhouette score
silhouette_average = silhouette_score(X_scaled, labels)
print("Silhouette Score is: ", silhouette_average)

### Cluster summaries

In [ ]:
# Calculate average amount, median amount, total amount, and top categories for each cluster

cluster_summary = df_clean.groupby('purpose_cluster').agg(
    avg_amount = ('Amount', 'mean'), 
    median_amount = ('Amount', 'median'), 
    total_amount = ('Amount', 'sum'), # <--- added total amount
    count = ('purpose_cluster', 'size')
)

In [ ]:
# Add top 2 for each type of Converted_ column per cluster
def get_top_n_columns(cluster_data, prefix, n=2):
    cols = [col for col in cluster_data.columns if col.startswith(prefix)]
    counts = cluster_data[cols].sum().sort_values(ascending=False)
    unique_top = []
    for col in counts.index:
        name = col.replace('Converted_', '')
        if name not in unique_top:
            unique_top.append(name)
        if len(unique_top) == n:
            break
    return ", ".join(unique_top)


top_categories = []
top_months = []
top_days = []
top_hours = []

for cluster in df_clean['purpose_cluster'].unique():
    cluster_data = df_clean[df_clean['purpose_cluster'] == cluster]
    top_categories.append(get_top_n_columns(cluster_data, 'Converted_Category'))
    top_months.append(get_top_n_columns(cluster_data, 'Converted_Month'))
    top_days.append(get_top_n_columns(cluster_data, 'Converted_DayOfWeek'))
    top_hours.append(get_top_n_columns(cluster_data, 'Converted_Hour'))

cluster_summary['top_categories'] = top_categories
cluster_summary['top_months'] = top_months
cluster_summary['top_days'] = top_days
cluster_summary['top_hours'] = top_hours

In [ ]:
cluster_summary

In [ ]:
# Remove 'Category_Other' from the top_categories column in the summary (so it doesn't show in the output)
cluster_summary['top_categories'] = cluster_summary['top_categories'].apply(
    lambda x: ", ".join([cat for cat in x.split(", ") if cat != "Category_Other"])
)

In [ ]:
# display cluster summary
print(cluster_summary)

In [ ]:
# Bar chart of Hour of the Day vs Total Amount Spent
amount_by_hour = df_clean.groupby('Hour')['Amount'].sum().reindex(range(24))
plt.figure(figsize=(10, 5))
sns.barplot(x=amount_by_hour.index, y=amount_by_hour.values, palette='viridis', hue=amount_by_hour.index, legend=False)
plt.title('Total Amount Spent by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Amount Spent')
plt.xticks(range(24))
plt.tight_layout()
plt.show()

In [ ]:
# Bar chart of Day of the Week vs Total Amount Spent
amount_by_day = df_clean.groupby('DayOfWeek')['Amount'].sum().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])
plt.figure(figsize=(8, 5))
sns.barplot(x=amount_by_day.index, y=amount_by_day.values, palette='viridis', hue=amount_by_day.index, legend=False)
plt.title('Total Amount Spent by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Amount Spent')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Bar chart of Category vs Total Amount Spent
amount_by_category = df_clean.groupby('Category')['Amount'].sum().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=amount_by_category.index, y=amount_by_category.values, palette='viridis', hue=amount_by_category.index, legend=False)
plt.title('Total Amount Spent by Category')
plt.xlabel('Category')
plt.ylabel('Total Amount Spent')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# remove the Other category from the list
amount_by_category.value_counts()